In [73]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

In [74]:
data = pd.read_csv("cars93.csv")

datax = data[["MPG.city", "MPG.highway", "Horsepower"]]
datax["MPG.city"] = (datax["MPG.city"] - datax["MPG.city"].min()) / (datax["MPG.city"].max() - datax["MPG.city"].min())
datax["MPG.highway"] = (datax["MPG.highway"] - datax["MPG.highway"].min()) / (datax["MPG.highway"].max() - datax["MPG.highway"].min())
datax["Horsepower"] = (datax["Horsepower"] - datax["Horsepower"].min()) / (datax["Horsepower"].max() - datax["Horsepower"].min())
datax = datax.to_numpy()
datay = data["Price"].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(datax, datay, random_state=1488, test_size=0.25)

print(x_test.shape)
print(y_test.shape)

(24, 3)
(24,)


/tmp/ipykernel_13832/3156339589.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datax["MPG.city"] = (datax["MPG.city"] - datax["MPG.city"].min()) / (datax["MPG.city"].max() - datax["MPG.city"].min())
/tmp/ipykernel_13832/3156339589.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datax["MPG.highway"] = (datax["MPG.highway"] - datax["MPG.highway"].min()) / (datax["MPG.highway"].max() - datax["MPG.highway"].min())
/tmp/ipykernel_13832/3156339589.py:6: SettingWithCopyWarning: 
A value is trying to be se

In [75]:
def predict(x, weights, b):
    return np.dot(x, weights)+b

def do_linear_regression(x_train, y_train):
    b = np.random.rand()
    weights = np.array([np.random.rand() for i in range(x_train.shape[1])])
    for _ in range(25):
        for i in range (x_train.shape[0]):
            pred = predict(x_train[i], weights, b)
            loss =(pred - y_train[i])**2
            
            dw = 2 * x_train[i] * (pred - y_train[i])
            db = 2 * (pred - y_train[i])
            
            weights -= 0.01 * dw
            b -= 0.01 * db
        print(loss)
    return b, weights

In [76]:
bias, weights = do_linear_regression(x_train, y_train)

46.35640325796516
73.38552525285559
67.39512572172238
57.55452550559082
48.86297803559553
41.72617940800743
35.917353876708034
31.177717990259712
27.292604233831185
24.09182033414428
21.44119712186034
19.234776689689664
17.3886009110233
15.835892516267275
14.523328331673323
13.408153976804156
12.455945344982707
11.63886761922544
10.934317602615065
10.323861902097974
9.792403922859972
9.327528215031053
8.918982618596148
8.558267753448561
8.238310366684287


In [77]:
predict(x_test, weights, bias)

array([14.32096051, 12.12013596, 18.69128589, 17.6726879 , 15.53304999,
       18.73440125, 14.67773086, 21.26090763,  8.71030139, 24.33756135,
       22.20013322, 23.39944066,  9.711933  , 19.35235834, 19.36206092,
       27.48853344, 12.61549078, 35.76648697, 22.62779278, 18.50023317,
       20.54207769, 25.69055957, 23.22105548, 15.66204638])

In [78]:
y_test

array([13.4, 10.3, 19.7, 10. , 14.1, 31.9, 11.4, 18.4,  7.4, 16.3, 17.7,
       26.3,  9.8, 20.2, 15.8, 19.3, 10. , 47.9, 19.1, 17.5, 19.8, 28. ,
       24.4, 11.1])

In [79]:
weights

array([-6.96198001, -5.35155521, 25.01259545])

In [80]:
bias

13.80585084718744

In [81]:
X_train = np.column_stack((np.ones(x_train.shape[0]), x_train))
X_test = np.column_stack((np.ones(x_test.shape[0]), x_test))

coefficients = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

y_pred_train = X_train @ coefficients

residuals = y_train - y_pred_train

n = len(y_train)
p = X_train.shape[1]

residual_variance = np.sum(residuals**2) / (n - p)

X_transpose_X_inv = np.linalg.inv(X_train.T @ X_train)
coeff_variance = residual_variance * X_transpose_X_inv

standard_errors = np.sqrt(np.diag(coeff_variance))

t_statistics = coefficients / standard_errors

p_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p)) for t in t_statistics]

for i, (coef, t_stat, p_val) in enumerate(zip(coefficients, t_statistics, p_values)):
    print(f"Коэффициент {i}: {coef:.4f}, t-статистика = {t_stat:.4f}, p-value = {p_val:.4f}")

alpha = 0.05
significant = [p < alpha for p in p_values]

for i, sig in enumerate(significant):
    if sig:
        print(f"Коэффициент {i} значим (p < {alpha})")
    else:
        print(f"Коэффициент {i} не значим (p >= {alpha})")


Коэффициент 0: 10.0914, t-статистика = 3.4046, p-value = 0.0011
Коэффициент 1: -1.9374, t-статистика = -0.1328, p-value = 0.8948
Коэффициент 2: -4.2660, t-статистика = -0.2974, p-value = 0.7671
Коэффициент 3: 31.4913, t-статистика = 6.8164, p-value = 0.0000
Коэффициент 0 значим (p < 0.05)
Коэффициент 1 не значим (p >= 0.05)
Коэффициент 2 не значим (p >= 0.05)
Коэффициент 3 значим (p < 0.05)
